# DataProfiler - Profilers

Data profiling is the process of examining a dataset and collecting statistical or informational summaries about said dataset.

The Profilers inside the DataProfiler is designed to calculate multiple statistics, make predictions on the entities inside a given column / key-value store (via the Labeler) and generally determine informational summaries. 

In [1]:
import os
import sys
import json
sys.path.insert(0, '..')
import dataprofiler as dp

data_path = "../dataprofiler/tests/data"

## Reporting

One of the primary purposes of the Profiler are to quickly identify what is in the dataset. This can be useful for analyzing a dataset prior to use or determining which columns could be useful for a given purpose.

In terms of reporting, there are multiple reporting options:

* **Pretty**: Floats are rounded to four decimal places, and lists are shortened.
* **Compact**: Similar to pretty, but removes detailed statistics such as runtimes, label probabilities, index locations of null types, etc.
* **Serializable**: Output is json serializable and not prettified
* **Flat**: Nested Output is returned as a flattened dictionary

The **Pretty** and **Compact** reports are the two most commonly used reports and includes `global_stats` and `data_stats` for the given dataset. 

`global_stats` contains overall properties of the data such as number of rows/columns, null ratio, duplicate ratio:

```
"global_stats": {
    "samples_used": int,
    "column_count": int,
    "row_count": int,
    "row_has_null_ratio": float,
    "row_is_null_ratio": float,    
    "unique_row_ratio": float,
    "duplicate_row_count": int,
    "file_type": string,
    "encoding": string,
}
```

`data_stats` contains specific properties and statistics for each column such as min, max, mean, variance, etc.


```
"data_stats": {
    <column name>: {
        "column_name": string,
        "data_type": string,
        "data_label": string,
        "categorical": bool,
        "order": string,
        "samples": list(str),
        "statistics": {
            "sample_size": int,
            "null_count": int,
            "null_types": list(string),
            "null_types_index": {
                string: list(int)
            },
            "data_type_representation": [string, list(string)],
            "min": [null, float],
            "max": [null, float],
            "mean": float,
            "variance": float,
            "stddev": float,
            "histogram": { 
                "bin_counts": list(int),
                "bin_edges": list(float),
            },
            "quantiles": {
                int: float
            }
            "vocab": list(char),
            "avg_predictions": dict(float), 
            "data_label_representation": dict(float),
            "categories": list(str),
            "unique_count": int,
            "unique_ratio": float,
            "precision": {
                'min': int,
                'max': int,
                'mean': float,
                'var': float,
                'std': float,
                'sample_size': int,
                'margin_of_error': float,
                'confidence_level': float		
            },
            "times": dict(float),
            "format": string
        }
    }
}
```

In the example, the `compact` format of the report is used to shorten the full list of the results. To get more results related to detailed predictions at the entity level from the DataLabeler component or histogram results, the format `pretty` should be used.

In [ ]:
data = dp.Data(os.path.join(data_path, "csv/aws_honeypot_marx_geo.csv"))
profile = dp.Profiler(data)

# Compact - A high level view, good for quick reviews
report  = profile.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))

It should be noted, in addition to reading the input data from multiple file types, DataProfiler allows the input data as a dataframe.

In [ ]:
# run data profiler and get the report
import pandas as pd
my_dataframe = pd.DataFrame([[1, 2.0],[1, 2.2],[-1, 3]], columns=["col_int", "col_float"])
profile = dp.Profiler(my_dataframe)

report  = profile.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))

## Profiler options

DataProfiler can run several selected components if needed. For example, if the users only want the statistics information, they may turn off the DataLabeler functionality. Below, let's remove the histogram and data labeler component while running DataProfiler. 

Full list of options: https://capitalone.github.io/DataProfiler

In [ ]:
profile_options = dp.ProfilerOptions()
profile_options.set({
    "histogram.is_enabled": False,
    "data_labeler.is_enabled": False
})

profile = dp.Profiler(data, profiler_options=profile_options)
report  = profile.report(report_options={"output_format":"compact"})

# Print the report
print(json.dumps(report, indent=4))

### Updating Profiles

Beyond just profiling, one of the unique aspects of the DataProfiler (as compared to pandas-profiling) is the ability to update the profiles. To update appropriately, the schema (columns / keys) must match appropriately.

In [ ]:
# Load and profile a CSV file
data = dp.Data(os.path.join(data_path, "csv/sparse-first-and-last-column-header-and-author.txt"))
profile = dp.Profiler(data)

# Update the profile with new data:
new_data = dp.Data(os.path.join(data_path, "csv/sparse-first-and-last-column-skip-header.txt"))
profile.update_profile(new_data)

# Report the compact version of the profile
report  = profile.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))

### Merging Profiles

Merging profiles are an alternative method for updating profiles. Particularly, multiple profiles can be generated seperately, then added together with a simple `+` command: `profile3 = profile1 + profile2`

In [ ]:
# Load a CSV file with a schema
data1 = dp.Data(os.path.join(data_path, "csv/sparse-first-and-last-column-header-and-author.txt"))
profile1 = dp.Profiler(data1)

# Load another CSV file with the same schema
data2 = dp.Data(os.path.join(data_path, "csv/sparse-first-and-last-column-skip-header.txt"))
profile2 = dp.Profiler(data2)

# Merge the profiles
profile3 = profile1 + profile2

# Report the compact version of the profile
report  = profile3.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))

As you can see, the `update_profile` function and the `+` operator function similarly. The reason the `+` operator is important is that it's possible to *save and load profiles*, which we cover next.

### Saving and Loading a Profile

Not only can the Profiler create and update profiles, it's also possible to save, load then manipulate profiles.

In [4]:
# Load data
data = dp.Data(os.path.join(data_path, "csv/iris.csv"))

# Generate a profile
profile = dp.Profiler(data)

# Save a profile to disk for later use
profile.save(filepath="my_profile.pkl")

# Load a profile from disk
loaded_profile = dp.Profiler.load("my_profile.pkl")

# Report the compact version of the profile
report = profile.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))

100%|██████████| 6/6 [00:00<00:00, 841.05it/s]

Finding the Null values in the columns... (with 3 processes)



  0%|          | 0/6 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


100%|██████████| 6/6 [00:00<00:00,  9.53it/s]


{
    "global_stats": {
        "samples_used": 150,
        "column_count": 6,
        "row_count": 150,
        "row_has_null_ratio": 0.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "csv",
        "encoding": "utf-8"
    },
    "data_stats": {
        "Id": {
            "column_name": "Id",
            "data_type": "int",
            "data_label": "INTEGER",
            "categorical": false,
            "order": "ascending",
            "samples": "['35', '140', '41', '137', '139']",
            "statistics": {
                "min": 1.0,
                "max": 150.0,
                "mean": 75.5,
                "variance": 1887.5,
                "stddev": 43.4454,
                "quantiles": {
                    "0": 38.0265,
                    "1": 75.1523,
                    "2": 112.9735
                },
                "unique_count": 150,
                "unique_ratio": 1.0,
                

With the ability to save and load profiles, profiles can be generated via multiple machines then merged. Further, profiles can be stored and later used in applications such as change point detction, synthetic data generaiton, and more. 

In [ ]:
# Load a multiple files via the Data class
filenames = ["csv/sparse-first-and-last-column-header-and-author.txt",
             "csv/sparse-first-and-last-column-skip-header.txt"]
data_objects = []
for filename in filenames:
    data_objects.append(dp.Data(os.path.join(data_path, filename)))


# Generate and save profiles
for i in range(len(data_objects)):
    profile = dp.Profiler(data_objects[i])
    profile.save(filepath="data-"+str(i)+".pkl")


# Load profiles and add them together
profile = None
for i in range(len(data_objects)):
    if profile is None:
        profile = dp.Profiler.load("data-"+str(i)+".pkl")
    else:
        profile += dp.Profiler.load("data-"+str(i)+".pkl")


# Report the compact version of the profile
report = profile.report(report_options={"output_format":"compact"})
print(json.dumps(report, indent=4))